# Creating the dataset

## Face extraction in fake video 

In [ ]:
import cv2
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
#from time import time
from facenet_pytorch.models.mtcnn import MTCNN
detector = MTCNN(margin=0,thresholds=[0.85, 0.95, 0.95])
from concurrent.futures import ThreadPoolExecutor
import cv2
import multiprocessing
import sys
import time

In [ ]:
path_mask = 'C:\\Users\\Himanshu\\Desktop\\manipulated_sequences\\DeepFakeDetection\\masks\\videos\\'

In [ ]:
names = os.listdir('Fake')
len(names)

In [ ]:
already_present = os.listdir('C:\\Users\\Himanshu\\Desktop\\fake')

In [ ]:
for i in already_present:
    try:
        names.remove(i)
    except ValueError: #if there is a file in already_present but not in Fake videos.
        continue

In [ ]:
def deepface(mask):
#     print('deepface')
    for i in range(mask.shape[0]):
        if(sum(mask[i,:,0])): 
            y_beg = i
            break
    for i in range(mask.shape[0]-1, 0 , -1):
        if(sum(mask[i,:,0])): 
            y_end = i
            break
    for i in range(mask.shape[1]):
        if(mask[y_beg,i,0]): 
            x_beg = i
            break
    for i in range(mask.shape[1]-1, 0 , -1):
        if(mask[y_beg,i,0]): 
            x_end = i
            break
    return ( x_beg, y_beg, x_end, y_end)

In [ ]:
#t = time()
def parallel(name):
    fake_vid = cv2.VideoCapture('Fake\\' + name)
    frame_cnt = fake_vid.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_cnt < 160: 
        print('less frame', end = ' ')
        return
    if frame_cnt > 160:
        if not os.path.exists('C:\\Users\\Himanshu\\Desktop\\fake\\' + name):
            os.makedirs('C:\\Users\\Himanshu\\Desktop\\fake\\' + name)
        fake_mask = cv2.VideoCapture(path_mask + '\\' + name)
        fake_mask.set(1, 100)
        fake_vid.set(1, 100)
        while_safety = 0
        j = 0
        while(j < 32):
            _, frame_vid = fake_vid.read()
            _, frame_mask = fake_mask.read()
            if (frame_vid is None) | (frame_mask is None):  # if we get a bad return flag or the image we read is None, lets not save
                while_safety += 1
                if while_safety == 50: 
                    return
                continue
            frame_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2RGB)
            result = deepface(frame_mask)
            frame_vid = cv2.cvtColor(frame_vid, cv2.COLOR_BGR2RGB)
            face_coor = detector.detect(frame_vid, landmarks=True)[0]
            min_dist = 100000
            ind = 0
            try:
                for i, data in enumerate(face_coor):
                    dist = sum(abs(data - result))
                    if dist < min_dist:
                        ind = i;
                        min_dist = dist
            except:
                while_safety += 1
                if while_safety == 50: 
                    return
                continue
            x_beg = int(face_coor[ind][0])
            x_end = int(face_coor[ind][2])
            y_beg = int(face_coor[ind][1])
            y_end = int(face_coor[ind][3])
            to_save = frame_vid[y_beg:y_end, x_beg:x_end, :]
            try:
                to_save = cv2.cvtColor(cv2.resize(to_save, (64, 64)), cv2.COLOR_BGR2RGB)
                cv2.imwrite('C:\\Users\\Himanshu\\Desktop\\fake\\' + name + '\\' + str(j) + '.jpg', to_save)
            except:
                return
            j+=1
        fake_mask.release()
    fake_vid.release()
    cv2.destroyAllWindows()

In [ ]:
for i in tqdm(range(4 + 335, len(names))):
    print(names[i])
    parallel(names[i])
    

## Extracting face in real videos

In [1]:
import cv2
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
#from time import time
from facenet_pytorch.models.mtcnn import MTCNN

In [2]:
detector = MTCNN(margin=0,thresholds=[0.85, 0.95, 0.95], keep_all = True, device = 0, image_size = 64)

In [3]:
names = os.listdir('Original')

In [4]:
path = 'C:\\Users\\Himanshu\\Desktop\\original\\'

In [17]:
def saving_face_original(name):
    vid = cv2.VideoCapture('Original\\' + name)
    frame_cnt = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_cnt < 160: 
        print('less frame', end = ' ')
        return
    if frame_cnt > 233:
        if not os.path.exists('C:\\Users\\Himanshu\\Desktop\\original\\' + name):
            os.makedirs('C:\\Users\\Himanshu\\Desktop\\original\\' + name)
        vid.set(1, 200)
        while_safety = 0
        j = 0
        while(j < 32):
            flag, frame = vid.read()
            if flag is False:
                while_safety += 1
                print('Error in ' + name)
                if while_safety == 50: 
                    break
                continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            _ = detector(frame, save_path= path + name + '\\'+ str(j) +'.jpg')
            j+=1
    vid.release()

In [18]:
for name in tqdm(names):
    saving_face_original(name)



  0%|                                                                                          | 0/363 [00:00<?, ?it/s]

  0%|▏                                                                                 | 1/363 [00:06<40:02,  6.64s/it]

  1%|▍                                                                                 | 2/363 [00:14<41:17,  6.86s/it]

  1%|▋                                                                                 | 3/363 [00:21<41:30,  6.92s/it]

  1%|▉                                                                                 | 4/363 [00:28<41:58,  7.02s/it]

  1%|█▏                                                                                | 5/363 [00:35<42:04,  7.05s/it]

  2%|█▎                                                                                | 6/363 [00:42<42:27,  7.14s/it]

  2%|█▌                                                                                | 7/363 [00:50<43:40,  7.36s/it]

  2%|█▊                       

 35%|████████████████████████████▏                                                   | 128/363 [15:50<28:02,  7.16s/it]

 36%|████████████████████████████▍                                                   | 129/363 [15:57<27:25,  7.03s/it]

 36%|████████████████████████████▋                                                   | 130/363 [16:05<28:22,  7.30s/it]

 36%|████████████████████████████▊                                                   | 131/363 [16:13<28:49,  7.45s/it]

 36%|█████████████████████████████                                                   | 132/363 [16:20<28:41,  7.45s/it]

 37%|█████████████████████████████▎                                                  | 133/363 [16:28<29:34,  7.71s/it]

 37%|█████████████████████████████▌                                                  | 134/363 [16:36<29:14,  7.66s/it]

 37%|█████████████████████████████▊                                                  | 135/363 [16:44<29:30,  7.76s/it]

 37%|███████████████████████████

less frame 



 48%|██████████████████████████████████████▏                                         | 173/363 [21:36<20:46,  6.56s/it]

 48%|██████████████████████████████████████▎                                         | 174/363 [21:44<21:55,  6.96s/it]

 48%|██████████████████████████████████████▌                                         | 175/363 [21:52<22:38,  7.22s/it]

 48%|██████████████████████████████████████▊                                         | 176/363 [21:59<22:42,  7.29s/it]

 49%|███████████████████████████████████████                                         | 177/363 [22:07<22:41,  7.32s/it]

 49%|███████████████████████████████████████▏                                        | 178/363 [22:15<23:32,  7.64s/it]

 49%|███████████████████████████████████████▍                                        | 179/363 [22:23<23:25,  7.64s/it]

 50%|███████████████████████████████████████▋                                        | 180/363 [22:30<23:09,  7.59s/it]

 50%|█████████████████████████

 83%|██████████████████████████████████████████████████████████████████▎             | 301/363 [38:11<07:54,  7.65s/it]

 83%|██████████████████████████████████████████████████████████████████▌             | 302/363 [38:19<07:52,  7.75s/it]

 83%|██████████████████████████████████████████████████████████████████▊             | 303/363 [38:28<08:00,  8.00s/it]

 84%|██████████████████████████████████████████████████████████████████▉             | 304/363 [38:36<07:46,  7.91s/it]

 84%|███████████████████████████████████████████████████████████████████▏            | 305/363 [38:43<07:26,  7.70s/it]

 84%|███████████████████████████████████████████████████████████████████▍            | 306/363 [38:50<07:12,  7.58s/it]

 85%|███████████████████████████████████████████████████████████████████▋            | 307/363 [38:58<07:11,  7.71s/it]

 85%|███████████████████████████████████████████████████████████████████▉            | 308/363 [39:05<06:55,  7.56s/it]

 85%|███████████████████████████